In [ ]:
# !pip install webdriver-manager selenium pytube

In [1]:
import re 
import time 
import random
import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

In [38]:
def scroll(driver, page_num=10):
    try:        
        last_page_height = driver.execute_script("return document.documentElement.scrollHeight")
        for _ in range(page_num):
            pause_time = random.uniform(1, 2)

            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
            
            time.sleep(pause_time)
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight-50)")
            
            time.sleep(pause_time)
            new_page_height = driver.execute_script("return document.documentElement.scrollHeight")

            if new_page_height == last_page_height:
                print("스크롤 완료")
                break
                
            else:
                last_page_height = new_page_height
            
    except Exception as e:
        print("에러 발생: ", e)

In [54]:
def text2time(x):
    x = ' '.join(x)
    if not str.isdigit(x[0][0]):
        x = x[1:] # example -> 전 3분 50초 / else -> 1시간 3분 50초
    t = re.sub('[^0-9]', ' ', x).split()
    outs = 0
    if len(t) == 2:
        for v, k in zip(t, [60, 1]):
            outs += int(v) * k
    
    elif len(t) == 3:
        for v, k in zip(t, [3600, 60, 1]):
            outs += int(v) * k 
    
    return outs

In [63]:
def crawling(name, page_num=10, threshold=3*60):
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)
    URL = f'https://www.youtube.com/user/{name}/videos'
    
    driver.get(URL)
    time.sleep(3)
    
    scroll(driver, page_num=page_num)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    source = soup.select('h3 > a')
    
    title, url, times = [], [], []

    for idx in source:
        t = text2time(idx.get('aria-label').split()[-3:]) 
        if t > threshold : # default -> 3 min 
            title.append(idx.text)
            url.append(URL + idx.get('href'))
            times.append(t)
            
    
    return {
        'title': title, 
        'url': url, 
        'time': times
    }

In [64]:
output = crawling('BBCNews')

In [65]:
output 

,title,url,time
0,Ukraine frontline: exhaustion of war in battle...,https://www.youtube.com/user/BBCNews/videos/wa...,234
1,Who could be Donald Trump's Vice President? | ...,https://www.youtube.com/user/BBCNews/videos/wa...,371
2,Vladimir Putin killed my husband says Alexei N...,https://www.youtube.com/user/BBCNews/videos/wa...,406
3,Alexei Navalny: Russian opposition leader’s mo...,https://www.youtube.com/user/BBCNews/videos/wa...,260
4,Israel indicates deadline for Gaza ground offe...,https://www.youtube.com/user/BBCNews/videos/wa...,371
...,...,...,...
67,King meets Prince Harry as he “steps back” fro...,https://www.youtube.com/user/BBCNews/videos/wa...,269
68,What’s next after Donald Trump's failed immuni...,https://www.youtube.com/user/BBCNews/videos/wa...,339
69,"Boeing: Bolts missing from plane door, says bl...",https://www.youtube.com/user/BBCNews/videos/wa...,206
70,Afghan special forces relocation cases face re...,https://www.youtube.com/user/BBCNews/videos/wa...,268
